# Gemini API: 安全快速指南


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Safety.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


Gemini API 具有可調整的安全設定。本筆記本將逐步指導你如何使用這些設定。你將會撰寫一個遭到封鎖的提示，找出原因，然後調整篩選器以解鎖提示。

安全性是一個重要的主題，你可以前往本筆記本結尾處的連結了解更多資訊。本文著重於程式碼。


In [ ]:
!pip install -q -U google-generativeai # Install the Python SDK

In [ ]:
import google.generativeai as genai

## 建立你的 API 金鑰

要執行以下Cell，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 密碼中。如果你還沒有 API 金鑰，或不確定如何建立 Colab 密碼，請參閱 [驗證](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) Quickstart 作為範例。


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 提示回饋

[Model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 方法回傳的結果為 [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/generativeai/types/GenerateContentResponse)。


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro')

unsafe_prompt =   # Put your unsafe prompt here
response = model.generate_content(unsafe_prompt)

此回應物件以兩種方式提供安全性回饋：

* `prompt_feedback.safety_ratings` 屬性包含輸入提示的安全評級清單。

* 如果你的提示遭封鎖，`prompt_feedback.block_reason` 欄位會說明原因。


In [ ]:
bool(response.prompt_feedback.block_reason)

In [ ]:
response.prompt_feedback.safety_ratings

如果提示被安全評分封鎖，你將不會在回應中獲得任何候選人：


In [ ]:
response.candidates

### 安全設定


調整安全設定，提示就不會再被封鎖。Gemini API 有四種可設定的安全設定。


In [ ]:
response = model.generate_content(
    unsafe_prompt,
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL' : 'BLOCK_NONE',
        'DANGEROUS' : 'BLOCK_NONE'
    })

有了新的設定，「blocked_reason」不再設定。


In [ ]:
bool(response.prompt_feedback.block_reason)

並回傳一個候選回覆。


In [ ]:
len(response.candidates)

你可查看 `response.text` 以取得回應。


In [ ]:
response.text

### 候選人評分


對於沒有被封鎖的提示，回應物件會包含一個 `候選` 物件清單 (現在只有 1 個)。每個候選都包含一個完成原因：


In [ ]:
candidate = response.candidates[0]
candidate.finish_reason

`FinishReason.STOP` 指示模型在正常情況下完成其輸出。

`FinishReason.SAFETY` 指示候選者的 `safety_ratings` 超過請求的 `safety_settings` 閾值。


In [ ]:
candidate.safety_ratings

## 進階學習

透過下列有關 [安全準則](https://ai.google.dev/docs/safety_guidance) 和 [安全設定](https://ai.google.dev/docs/safety_setting_gemini) 的文章瞭解更多。

## 有用的 API 參考

4 個可設定的 Gemini API 安全設定：

* `HARM_CATEGORY_DANGEROUS`
*`HARM_CATEGORY_HARASSMENT`
* `HARM_CATEGORY_SEXUALLY_EXPLICIT`
* `HARM_CATEGORY_DANGEROUS`

注意：雖然 API [參考](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 包含其他，但其餘用於舊版模型。

* 你可以使用完整名稱或別名 (例如上述 Python 程式碼中所使用的 `DANGEROUS`) 來參考安全設定。

安全性設定可在 [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 建構函式中設定。

* 它們也可以在每個 [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 或 [ChatSession.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession?hl=en#send_message) 請求中傳遞。

- [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) 會傳回 [Prompt](https://ai.google.dev/api/python/google/ai/generativelanguage/Prompt) 的 [安全評級](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating)，並在 `safety_ratings` 屬性中傳回每個 [候選項目](https://ai.google.dev/api/python/google/ai/generativelanguage/Candidate) 的安全評級。

- [glm.SafetySetting](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetySetting) 包含：[glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 和 [glm.HarmBlockThreshold](https://ai.google.dev/api/python/google/generativeai/types/HarmBlockThreshold)

- [glm.SafetyRating](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) 包含 [HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 和 [HarmProbability](https://ai.google.dev/api/python/google/generativeai/types/HarmProbability)

[glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 枚舉同時包含 PaLM 和 Gemini 模型的類別。

- 在指定枚舉值時，SDK 將接受枚舉值本身，或它們的整數或字串表示。

- SDK 也會接受縮寫字串表示：`["HARM_CATEGORY_DANGEROUS_CONTENT", "DANGEROUS_CONTENT", "DANGEROUS"]` 全都合法。字串不分大小寫。
